In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
# Load the cleaned dataset
file_path = './cleaned_tweets.csv'
tweets_data = pd.read_csv(file_path)

# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    tweets_data['cleaned_text'], 
    tweets_data['airline_sentiment'], 
    test_size=0.1, 
    random_state=3, 
    stratify=tweets_data['airline_sentiment']  # Ensure class distribution is preserved
)

# Print class distribution
print("Training set class distribution:")
print(train_labels.value_counts())
print("\nTest set class distribution:")
print(test_labels.value_counts())


# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000)

# Fit and transform the training data
X_train = tfidf.fit_transform(train_data)
X_test = tfidf.transform(test_data)


# Initialize the model
svm_model = SGDClassifier(
    loss="hinge",
    penalty="l2",
    alpha=1e-4,
    max_iter=100,
    tol=None,
    shuffle=True,
    random_state=3
)

# Train the model
svm_model.fit(X_train, train_labels)


# Perform 10-fold cross-validation
# cv_scores = cross_val_score(svm_model, X_train, train_labels, cv=10, scoring="accuracy")
# Perform 10-fold cross-validation
cv_scores = cross_val_score(svm_model, X_train, train_labels, cv=10, scoring="accuracy")

# Calculate fold statistics
mean_accuracy = cv_scores.mean()
min_accuracy = cv_scores.min()
max_accuracy = cv_scores.max()
std_accuracy = cv_scores.std()

# Output cross-validation statistics
print("10-Fold Cross-Validation Results:")
print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"Minimum Accuracy: {min_accuracy:.4f}")
print(f"Maximum Accuracy: {max_accuracy:.4f}")
print(f"Standard Deviation: {std_accuracy:.4f}")

# Calculate mean validation accuracy
validation_accuracy = cv_scores.mean()
print(f"10-Fold Cross-Validation Accuracy: {validation_accuracy:.4f}")

# Calculate mean validation accuracy
validation_accuracy = cv_scores.mean()
print(f"10-Fold Cross-Validation Accuracy: {validation_accuracy:.4f}")
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Predict on test data
test_predictions = svm_model.predict(X_test)

# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Set Accuracy: {test_accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(test_labels, test_predictions))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(test_labels, test_predictions))


Training set class distribution:
negative    8170
neutral     2663
positive    1995
Name: airline_sentiment, dtype: int64

Test set class distribution:
negative    908
neutral     296
positive    222
Name: airline_sentiment, dtype: int64
10-Fold Cross-Validation Results:
Mean Accuracy: 0.8009
Minimum Accuracy: 0.7847
Maximum Accuracy: 0.8168
Standard Deviation: 0.0088
10-Fold Cross-Validation Accuracy: 0.8009
10-Fold Cross-Validation Accuracy: 0.8009
Test Set Accuracy: 0.8114

Classification Report:
              precision    recall  f1-score   support

    negative       0.84      0.94      0.89       908
     neutral       0.68      0.51      0.58       296
    positive       0.81      0.69      0.75       222

    accuracy                           0.81      1426
   macro avg       0.78      0.71      0.74      1426
weighted avg       0.80      0.81      0.80      1426


Confusion Matrix:
[[851  44  13]
 [120 152  24]
 [ 40  28 154]]


# Optuna Find the Best Combination 

In [3]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # 確保多語言支持


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import re
from nltk.stem import WordNetLemmatizer
import pandas as pd

# Initialize lemmatizer and contractions
lemmatizer = WordNetLemmatizer()
contractions = {"don't": "do not", "can't": "cannot", "i'm": "i am"}

# Define a cleaning function with toggleable options
def clean_text_variant(
    text, remove_mentions=True, remove_urls=True, lemmatize=True, expand_contractions=True
):
    text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", "", text)  # Remove emails
    text = re.sub(r"\$\d+(?:\.\d{2})?", "", text)  # Remove currency
    text = re.sub(r"[^\w\s,]", "", text, flags=re.UNICODE)  # Remove emojis
    text = re.sub(r"&[a-z]+;", "", text)  # Remove HTML escaped chars
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"[\u4e00-\u9fff]", "", text)  # Remove Chinese characters
    if remove_mentions:
        text = re.sub(r"@\w+", "", text)  # Remove mentions
    if remove_urls:
        text = re.sub(r"http[s]?://\S+", "", text)  # Remove URLs
    if expand_contractions:
        text = " ".join([contractions[word] if word in contractions else word for word in text.split()])
    text = re.sub(r"(.)\1{2,}", r"\1", text)  # Remove repeated characters
    text = " ".join([word for word in text.split() if 2 <= len(word) <= 15])  # Limit word length
    if lemmatize:
        text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])  # Lemmatize
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

# Load dataset
file_path = './cleaned_tweets.csv'
tweets_data = pd.read_csv(file_path)

# Store detailed results for all combinations
detailed_results = []

def objective(trial):
    # Suggest cleaning steps
    remove_mentions = trial.suggest_categorical("remove_mentions", [True, False])
    remove_urls = trial.suggest_categorical("remove_urls", [True, False])
    lemmatize = trial.suggest_categorical("lemmatize", [True, False])
    expand_contractions = trial.suggest_categorical("expand_contractions", [True, False])
    
    # Apply cleaning with the suggested configuration
    tweets_data['variant_cleaned_text'] = tweets_data['text'].apply(
        lambda x: clean_text_variant(
            x,
            remove_mentions=remove_mentions,
            remove_urls=remove_urls,
            lemmatize=lemmatize,
            expand_contractions=expand_contractions,
        )
    )
    
    # TF-IDF vectorization
    tfidf = TfidfVectorizer(max_features=5000)
    X = tfidf.fit_transform(tweets_data['variant_cleaned_text'])
    y = tweets_data['airline_sentiment']
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=3, stratify=y
    )
    
    # Train a model
    svm_model = SGDClassifier(
        loss="hinge", penalty="l2", alpha=1e-4, max_iter=100, tol=None, shuffle=True, random_state=3
    )
    svm_model.fit(X_train, y_train)
    
    # Perform 10-fold cross-validation
    cv_scores = cross_val_score(svm_model, X_train, y_train, cv=10, scoring="accuracy")
    mean_accuracy = cv_scores.mean()
    min_accuracy = cv_scores.min()
    max_accuracy = cv_scores.max()
    std_accuracy = cv_scores.std()

    # Print cross-validation details (optional for debugging)
    print(f"Cross-Validation Accuracy - Mean: {mean_accuracy:.4f}, Min: {min_accuracy:.4f}, Max: {max_accuracy:.4f}, Std: {std_accuracy:.4f}")
    
    # Evaluate on the test set
    y_pred = svm_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    confusion = confusion_matrix(y_test, y_pred)
    
    # Store detailed results
    detailed_results.append({
        "trial_params": trial.params,
        "cross_val_accuracy": mean_accuracy,
        "min_accuracy": min_accuracy,
        "max_accuracy": max_accuracy,
        "std_accuracy": std_accuracy,
        "test_accuracy": test_accuracy,
        "classification_report": report,
        "confusion_matrix": confusion,
    })
    
    return mean_accuracy

# Use Optuna to optimize
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=256, n_jobs=-1)

# Display the best results
best_trial = study.best_trial
best_params = best_trial.params

print("\nBest Cleaning Steps:")
print(best_params)
print(f"Best Cross-Validation Accuracy: {study.best_value:.4f}")

# Extract the best trial's detailed results
best_result = next(
    res for res in detailed_results if res["trial_params"] == best_params
)

# Add statistics for cross-validation accuracy range
print("\nCross-Validation Accuracy Details for Best Cleaning Steps:")
print(f"Mean Accuracy: {best_result['cross_val_accuracy']:.4f}")
print(f"Min Accuracy: {best_result['min_accuracy']:.4f}")
print(f"Max Accuracy: {best_result['max_accuracy']:.4f}")
print(f"Std Dev of Accuracy: {best_result['std_accuracy']:.4f}")

# Display the best test set results
print("\nBest Test Set Accuracy:")
print(best_result["test_accuracy"])
print("\nBest Classification Report:")
print(pd.DataFrame(best_result["classification_report"]).T)
print("\nBest Confusion Matrix:")
print(best_result["confusion_matrix"])



[I 2024-12-06 17:10:27,819] A new study created in memory with name: no-name-fc3f7395-7707-47c1-ad71-9f69829f5125


Cross-Validation Accuracy - Mean: 0.7993, Min: 0.7902, Max: 0.8137, Std: 0.0077
Cross-Validation Accuracy - Mean: 0.7993, Min: 0.7902, Max: 0.8137, Std: 0.0077


[I 2024-12-06 17:11:41,412] Trial 1 finished with value: 0.7992660532609925 and parameters: {'remove_mentions': False, 'remove_urls': False, 'lemmatize': False, 'expand_contractions': True}. Best is trial 1 with value: 0.7992660532609925.
[I 2024-12-06 17:11:42,679] Trial 3 finished with value: 0.7992660532609925 and parameters: {'remove_mentions': False, 'remove_urls': True, 'lemmatize': False, 'expand_contractions': False}. Best is trial 1 with value: 0.7992660532609925.


Cross-Validation Accuracy - Mean: 0.7993, Min: 0.7902, Max: 0.8137, Std: 0.0077
Cross-Validation Accuracy - Mean: 0.7993, Min: 0.7902, Max: 0.8137, Std: 0.0077
Cross-Validation Accuracy - Mean: 0.7993, Min: 0.7902, Max: 0.8137, Std: 0.0077


[I 2024-12-06 17:11:47,910] Trial 2 finished with value: 0.7992660532609925 and parameters: {'remove_mentions': False, 'remove_urls': True, 'lemmatize': False, 'expand_contractions': True}. Best is trial 1 with value: 0.7992660532609925.


...

Cross-Validation Accuracy - Mean: 0.8025, Min: 0.7894, Max: 0.8168, Std: 0.0091


[I 2024-12-06 17:31:37,547] Trial 252 finished with value: 0.8025395700161599 and parameters: {'remove_mentions': True, 'remove_urls': True, 'lemmatize': True, 'expand_contractions': True}. Best is trial 5 with value: 0.8025395700161599.


Cross-Validation Accuracy - Mean: 0.8025, Min: 0.7894, Max: 0.8168, Std: 0.0091
Cross-Validation Accuracy - Mean: 0.8025, Min: 0.7894, Max: 0.8168, Std: 0.0091
Cross-Validation Accuracy - Mean: 0.8025, Min: 0.7894, Max: 0.8168, Std: 0.0091


[I 2024-12-06 17:31:37,718] Trial 253 finished with value: 0.8025395700161599 and parameters: {'remove_mentions': True, 'remove_urls': True, 'lemmatize': True, 'expand_contractions': True}. Best is trial 5 with value: 0.8025395700161599.
[I 2024-12-06 17:31:37,772] Trial 254 finished with value: 0.8025395700161599 and parameters: {'remove_mentions': True, 'remove_urls': True, 'lemmatize': True, 'expand_contractions': True}. Best is trial 5 with value: 0.8025395700161599.
[I 2024-12-06 17:31:39,421] Trial 255 finished with value: 0.8025395700161599 and parameters: {'remove_mentions': True, 'remove_urls': True, 'lemmatize': True, 'expand_contractions': True}. Best is trial 5 with value: 0.8025395700161599.


Cross-Validation Accuracy - Mean: 0.8025, Min: 0.7894, Max: 0.8168, Std: 0.0091

Best Cleaning Steps:
{'remove_mentions': False, 'remove_urls': True, 'lemmatize': True, 'expand_contractions': True}
Best Cross-Validation Accuracy: 0.8025

Cross-Validation Accuracy Details for Best Cleaning Steps:
Mean Accuracy: 0.8025
Min Accuracy: 0.7894
Max Accuracy: 0.8168
Std Dev of Accuracy: 0.0091

Best Test Set Accuracy:
0.8092566619915849

Best Classification Report:
              precision    recall  f1-score      support
negative       0.841223  0.939427  0.887617   908.000000
neutral        0.672566  0.513514  0.582375   296.000000
positive       0.801075  0.671171  0.730392   222.000000
accuracy       0.809257  0.809257  0.809257     0.809257
macro avg      0.771622  0.708037  0.733462  1426.000000
weighted avg   0.799964  0.809257  0.799780  1426.000000

Best Confusion Matrix:
[[853  43  12]
 [119 152  25]
 [ 42  31 149]]


# Best Combination 

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer

# Initialize lemmatizer and contractions
lemmatizer = WordNetLemmatizer()
contractions = {"don't": "do not", "can't": "cannot", "i'm": "i am"}

# Define the cleaning function with the best parameters
def clean_text_variant(
    text, remove_mentions=False, remove_urls=True, lemmatize=True, expand_contractions=False
):
    if remove_mentions:
        text = re.sub(r"@\w+", "", text)  # Remove mentions
    if remove_urls:
        text = re.sub(r"http[s]?://\S+", "", text)  # Remove URLs
    text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", "", text)  # Remove emails
    text = re.sub(r"\$\d+(?:\.\d{2})?", "", text)  # Remove currency
    text = re.sub(r"[^\w\s,]", "", text, flags=re.UNICODE)  # Remove emojis
    text = re.sub(r"&[a-z]+;", "", text)  # Remove HTML escaped chars
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"[\u4e00-\u9fff]", "", text)  # Remove Chinese characters
    if expand_contractions:
        text = " ".join([contractions[word] if word in contractions else word for word in text.split()])
    text = re.sub(r"(.)\1{2,}", r"\1", text)  # Remove repeated characters
    text = " ".join([word for word in text.split() if 2 <= len(word) <= 15])  # Limit word length
    if lemmatize:
        text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])  # Lemmatize
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

# Load dataset
file_path = './cleaned_tweets.csv'
tweets_data = pd.read_csv(file_path)

# Apply the best cleaning steps
tweets_data['cleaned_text'] = tweets_data['text'].apply(clean_text_variant)

# TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(tweets_data['cleaned_text'])
y = tweets_data['airline_sentiment']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=3, stratify=y
)

# Train a model
svm_model = SGDClassifier(
    loss="hinge", penalty="l2", alpha=1e-4, max_iter=100, tol=None, shuffle=True, random_state=3
)
svm_model.fit(X_train, y_train)

# Evaluate on the test set
y_pred = svm_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=False)
confusion = confusion_matrix(y_test, y_pred)

# Output results
print(f"Test Set Accuracy: {test_accuracy:.6f}\n")
print("Classification Report:")
print(report)
print("\nConfusion Matrix:")
print(confusion)


Test Set Accuracy: 0.809958

Classification Report:
              precision    recall  f1-score   support

    negative       0.84      0.94      0.89       908
     neutral       0.67      0.51      0.58       296
    positive       0.81      0.68      0.74       222

    accuracy                           0.81      1426
   macro avg       0.77      0.71      0.73      1426
weighted avg       0.80      0.81      0.80      1426


Confusion Matrix:
[[853  44  11]
 [119 152  25]
 [ 41  31 150]]
